In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# input_data data files are available in the read-only "../input_data/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input_data directory

import os
for dirname, _, filenames in os.walk('/kaggle/input_data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
# import tensorflow_addons as tfa
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sn

from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split, GroupShuffleSplit 

import glob
import sys
import os
import math
import gc
import sys
import sklearn
import scipy

2024-06-02 09:37:03.543997: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-02 09:37:03.544045: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-02 09:37:03.545750: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# EduSign

In [3]:
train = pd.read_csv('/kaggle/input/asl-signs/train.csv')
len(train)

94477

In [4]:
display(train.head(5))
display(train.info())

,path,participant_id,sequence_id,sign
0,train_landmark_files/26734/1000035562.parquet,26734,1000035562,blow
1,train_landmark_files/28656/1000106739.parquet,28656,1000106739,wait
2,train_landmark_files/16069/100015657.parquet,16069,100015657,cloud
3,train_landmark_files/25571/1000210073.parquet,25571,1000210073,bird
4,train_landmark_files/62590/1000240708.parquet,62590,1000240708,owie


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94477 entries, 0 to 94476
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   path            94477 non-null  object
 1   participant_id  94477 non-null  int64 
 2   sequence_id     94477 non-null  int64 
 3   sign            94477 non-null  object
dtypes: int64(2), object(2)
memory usage: 2.9+ MB


None

# CONFIG

In [5]:
TOTAL_LANDMARKS = 543 #33 Pose, 468 Face , 21 Hand
NUM_CLASS = 250
VERBOSE = 2
PAD = -100
N_DIMS = 2
N_SAMPLES = len(train)
SEED = 102

# LANDMARK (BECAUSE THE DATASET USING OLD MEDIAPIPE, THERE WILL BE RIGHT HAND AND LEFT HAND)
USED_LANDMARK = ['left_hand', 'pose', 'right_hand']
START_INDICES = 468
LIPS_INDICES = [61, 185, 40, 39, 37, 0, 267, 269, 270, 409,
        291, 146, 91, 181, 84, 17, 314, 405, 321, 375,
        78, 191, 80, 81, 82, 13, 312, 311, 310, 415,
        95, 88, 178, 87, 14, 317, 402, 318, 324, 308, ]

LEFT_HAND_INDICES = np.arange(468,489)
RIGHT_HAND_INDICES = np.arange(522,543)
LEFT_POSE_INDICES = np.array([502, 504, 506, 508, 510])
RIGHT_POSE_INDICES = np.array([503, 505, 507, 509, 511])
INDICES_LEFT_DOMINANT = np.concatenate((LIPS_INDICES, LEFT_HAND_INDICES, LEFT_POSE_INDICES))
INDICES_RIGHT_DOMINANT  = np.concatenate((LIPS_INDICES, RIGHT_HAND_INDICES, RIGHT_POSE_INDICES))
HANDS_LANDMARK = np.concatenate((LEFT_HAND_INDICES, RIGHT_HAND_INDICES), axis = 0)
N_COLS = INDICES_LEFT_DOMINANT.size

LIPS_IDXS = np.argwhere(np.isin(INDICES_LEFT_DOMINANT, LIPS_INDICES)).squeeze()
LEFT_HAND_IDXS = np.argwhere(np.isin(INDICES_LEFT_DOMINANT, LEFT_HAND_INDICES)).squeeze()
RIGHT_HAND_IDXS = np.argwhere(np.isin(INDICES_LEFT_DOMINANT, RIGHT_HAND_INDICES)).squeeze()
HAND_IDXS = np.argwhere(np.isin(INDICES_LEFT_DOMINANT, HANDS_LANDMARK)).squeeze()
POSE_IDXS = np.argwhere(np.isin(INDICES_LEFT_DOMINANT, LEFT_POSE_INDICES)).squeeze()

#FOR TRAINING
BATCH_ALL_SIGNS_N = 4
INPUT_SIZE = 64 #How Many Frame
BACTCH_SIZE = 256
N_EPOCHS = 1000
LR = 1e-4
MASK_VAL = 4237

In [6]:
def get_file_path(path):
    return f'/kaggle/input/asl-signs/{path}'

train['file_path'] = train['path'].apply(get_file_path)
display(train.head(5))

,path,participant_id,sequence_id,sign,file_path
0,train_landmark_files/26734/1000035562.parquet,26734,1000035562,blow,/kaggle/input/asl-signs/train_landmark_files/2...
1,train_landmark_files/28656/1000106739.parquet,28656,1000106739,wait,/kaggle/input/asl-signs/train_landmark_files/2...
2,train_landmark_files/16069/100015657.parquet,16069,100015657,cloud,/kaggle/input/asl-signs/train_landmark_files/1...
3,train_landmark_files/25571/1000210073.parquet,25571,1000210073,bird,/kaggle/input/asl-signs/train_landmark_files/2...
4,train_landmark_files/62590/1000240708.parquet,62590,1000240708,owie,/kaggle/input/asl-signs/train_landmark_files/6...


# Encode the label

In [7]:
train['ord_value'] = train['sign'].astype('category').cat.codes
display(train)

,path,participant_id,sequence_id,sign,file_path,ord_value
0,train_landmark_files/26734/1000035562.parquet,26734,1000035562,blow,/kaggle/input/asl-signs/train_landmark_files/2...,25
1,train_landmark_files/28656/1000106739.parquet,28656,1000106739,wait,/kaggle/input/asl-signs/train_landmark_files/2...,232
2,train_landmark_files/16069/100015657.parquet,16069,100015657,cloud,/kaggle/input/asl-signs/train_landmark_files/1...,48
3,train_landmark_files/25571/1000210073.parquet,25571,1000210073,bird,/kaggle/input/asl-signs/train_landmark_files/2...,23
4,train_landmark_files/62590/1000240708.parquet,62590,1000240708,owie,/kaggle/input/asl-signs/train_landmark_files/6...,164
...,...,...,...,...,...,...
94472,train_landmark_files/53618/999786174.parquet,53618,999786174,white,/kaggle/input/asl-signs/train_landmark_files/5...,238
94473,train_landmark_files/26734/999799849.parquet,26734,999799849,have,/kaggle/input/asl-signs/train_landmark_files/2...,108
94474,train_landmark_files/25571/999833418.parquet,25571,999833418,flower,/kaggle/input/asl-signs/train_landmark_files/2...,86
94475,train_landmark_files/29302/999895257.parquet,29302,999895257,room,/kaggle/input/asl-signs/train_landmark_files/2...,188


In [8]:
# Dictionaries to translate sign <-> ordinal encoded sign
SIGN2ORD = train[['sign', 'ord_value']].set_index('sign').squeeze().to_dict()
ORD2SIGN = train[['ord_value', 'sign']].set_index('ord_value').squeeze().to_dict()

In [9]:
SIGN2ORD

{'blow': 25,
 'wait': 232,
 'cloud': 48,
 'bird': 23,
 'owie': 164,
 'duck': 67,
 'minemy': 143,
 'lips': 134,
 'flower': 86,
 'time': 220,
 'vacuum': 231,
 'apple': 8,
 'puzzle': 180,
 'mitten': 144,
 'there': 216,
 'dry': 65,
 'shirt': 195,
 'owl': 165,
 'yellow': 243,
 'not': 156,
 'zipper': 249,
 'clean': 45,
 'closet': 47,
 'quiet': 181,
 'have': 108,
 'brother': 30,
 'clown': 49,
 'cheek': 41,
 'cute': 54,
 'store': 207,
 'shoe': 196,
 'wet': 235,
 'see': 193,
 'empty': 70,
 'fall': 74,
 'balloon': 14,
 'frenchfries': 89,
 'finger': 80,
 'same': 190,
 'cry': 52,
 'hungry': 121,
 'orange': 162,
 'milk': 142,
 'go': 97,
 'drawer': 62,
 'TV': 0,
 'another': 6,
 'giraffe': 93,
 'wake': 233,
 'bee': 19,
 'bad': 13,
 'can': 35,
 'say': 191,
 'callonphone': 34,
 'finish': 81,
 'old': 159,
 'backyard': 12,
 'sick': 198,
 'look': 136,
 'that': 215,
 'black': 24,
 'yourself': 246,
 'open': 161,
 'alligator': 4,
 'moon': 146,
 'find': 78,
 'pizza': 172,
 'shhh': 194,
 'fast': 76,
 'jacket':

In [10]:
ORD2SIGN

{25: 'blow',
 232: 'wait',
 48: 'cloud',
 23: 'bird',
 164: 'owie',
 67: 'duck',
 143: 'minemy',
 134: 'lips',
 86: 'flower',
 220: 'time',
 231: 'vacuum',
 8: 'apple',
 180: 'puzzle',
 144: 'mitten',
 216: 'there',
 65: 'dry',
 195: 'shirt',
 165: 'owl',
 243: 'yellow',
 156: 'not',
 249: 'zipper',
 45: 'clean',
 47: 'closet',
 181: 'quiet',
 108: 'have',
 30: 'brother',
 49: 'clown',
 41: 'cheek',
 54: 'cute',
 207: 'store',
 196: 'shoe',
 235: 'wet',
 193: 'see',
 70: 'empty',
 74: 'fall',
 14: 'balloon',
 89: 'frenchfries',
 80: 'finger',
 190: 'same',
 52: 'cry',
 121: 'hungry',
 162: 'orange',
 142: 'milk',
 97: 'go',
 62: 'drawer',
 0: 'TV',
 6: 'another',
 93: 'giraffe',
 233: 'wake',
 19: 'bee',
 13: 'bad',
 35: 'can',
 191: 'say',
 34: 'callonphone',
 81: 'finish',
 159: 'old',
 12: 'backyard',
 198: 'sick',
 136: 'look',
 215: 'that',
 24: 'black',
 246: 'yourself',
 161: 'open',
 4: 'alligator',
 146: 'moon',
 78: 'find',
 172: 'pizza',
 194: 'shhh',
 76: 'fast',
 125: 'jac

# Load Relevant Data

In [ ]:
def load_relevant_data_subset(pq_path):
    #data_columns = ['x','y','z'] #IF USING Z VALUE BUT USUALY WE DONT
    data_columns = ['x','y']
    data = pd.read_parquet(pq_path, columns = data_columns)
    n_frames = int(len(data) / TOTAL_LANDMARKS)
    data = data.values.reshape(n_frames, TOTAL_LANDMARKS, len(data_columns))
    return data.astype(np.float32)

## Data Preprocessing

In [ ]:
class PreprocessLayer(tf.keras.layers.Layer):
    def __init__(self):
        super(PreprocessLayer, self).__init__()
        normalisation_correction = tf.constant([
                [0] * len(LIPS_INDICES) + [0.50] * len(LEFT_HAND_INDICES) + [0.50] * len(POSE_IDXS),
                [0] * len(INDICES_LEFT_DOMINANT),
                #[0] * len(LANDMARK_IDXS_LEFT_DOMINANT0),#Uncomment when using Z Value
            ],
            dtype=tf.float32,
        )
        self.normalisation_correction = tf.transpose(normalisation_correction, [1, 0])

    def padding_edge(self, t, repeats, side):
        if side == 'LEFT':
            return tf.concat([tf.repeat(t[:1], repeats=repeats, axis=0), t], axis=0)
        elif side == 'RIGHT':
            return tf.concat([t, tf.repeat(t[-1:], repeats=repeats, axis=0)], axis=0)

    @tf.function(input_signature=[tf.TensorSpec(shape=[None, TOTAL_LANDMARKS, N_DIMS], dtype=tf.float32)])
    def call(self, input_data):
        N_FRAMES0 = tf.shape(input_data)[0]

        left_hand_sum = tf.reduce_sum(tf.where(tf.math.is_nan(tf.gather(input_data, LEFT_HAND_INDICES, axis=1)), 0, 1))
        right_hand_sum = tf.reduce_sum(tf.where(tf.math.is_nan(tf.gather(input_data, RIGHT_HAND_INDICES, axis=1)), 0, 1))
        left_dominant = left_hand_sum >= right_hand_sum

        if left_dominant:
            dominant_hand_idxs = LEFT_HAND_INDICES
        else:
            dominant_hand_idxs = RIGHT_HAND_INDICES
            
        frames_hands_non_nan_sum = tf.math.reduce_sum(
             tf.where(tf.math.is_nan(tf.gather(input_data, dominant_hand_idxs, axis=1)), 0, 1),
             axis=[1, 2]
         )    
        
        non_empty_frames_idxs = tf.where(frames_hands_non_nan_sum > 0)
        non_empty_frames_idxs = tf.squeeze(non_empty_frames_idxs, axis=1)
        data = tf.gather(input_data, non_empty_frames_idxs, axis=0)

        non_empty_frames_idxs = tf.cast(non_empty_frames_idxs, tf.float32)
        non_empty_frames_idxs -= tf.reduce_min(non_empty_frames_idxs)

        N_FRAMES = tf.shape(data)[0]

        if left_dominant:
            data = tf.gather(data, INDICES_LEFT_DOMINANT, axis=1)
        else:
            data = tf.gather(data, INDICES_RIGHT_DOMINANT, axis=1)
            data = self.normalisation_correction + (
                (data - self.normalisation_correction) * tf.where(self.normalisation_correction != 0, -1.0, 1.0))

        if N_FRAMES < INPUT_SIZE:
            pad_size = INPUT_SIZE - N_FRAMES
            non_empty_frames_idxs = tf.pad(non_empty_frames_idxs, [[0, pad_size]], constant_values=-1)
            data = tf.pad(data, [[0, pad_size], [0, 0], [0, 0]], constant_values=0)
            data = tf.where(tf.math.is_nan(data), 0.0, data)
            return data, non_empty_frames_idxs
        else:
            if N_FRAMES < INPUT_SIZE ** 2:
                repeats = tf.math.floordiv(INPUT_SIZE * INPUT_SIZE, N_FRAMES0)
                data = tf.repeat(data, repeats=repeats, axis=0)
                non_empty_frames_idxs = tf.repeat(non_empty_frames_idxs, repeats=repeats, axis=0)

            pool_size = tf.math.floordiv(len(data), INPUT_SIZE)
            if tf.math.mod(len(data), INPUT_SIZE) > 0:
                pool_size += 1

            pad_size = (pool_size * INPUT_SIZE) - len(data) if pool_size == 1 else (pool_size * INPUT_SIZE) % len(data)
            pad_left = tf.math.floordiv(pad_size, 2) + tf.math.floordiv(INPUT_SIZE, 2)
            pad_right = pad_left + 1 if tf.math.mod(pad_size, 2) > 0 else pad_left

            data = self.padding_edge(data, pad_left, 'LEFT')
            data = self.padding_edge(data, pad_right, 'RIGHT')
            non_empty_frames_idxs = self.padding_edge(non_empty_frames_idxs, pad_left, 'LEFT')
            non_empty_frames_idxs = self.padding_edge(non_empty_frames_idxs, pad_right, 'RIGHT')

            data = tf.reshape(data, [INPUT_SIZE, -1, N_COLS, N_DIMS])
            non_empty_frames_idxs = tf.reshape(non_empty_frames_idxs, [INPUT_SIZE, -1])
            
            #
            data = tf.reduce_mean(data, axis=1)
            non_empty_frames_idxs = tf.reduce_mean(non_empty_frames_idxs, axis=1)

            data = tf.where(tf.math.is_nan(data), 0.0, data)
            
            return data, non_empty_frames_idxs

preprocess_layer = PreprocessLayer()


In [ ]:
def get_data_preprocess(file_path):
    data = load_relevant_data_subset(file_path)
    data = preprocess_layer(data)    
    return data

In [ ]:
def initialize_arrays(N_SAMPLES, INPUT_SIZE, N_COLS, N_DIMS):
    X = np.zeros([N_SAMPLES, INPUT_SIZE, N_COLS, N_DIMS], dtype=np.float32)
    y = np.zeros([N_SAMPLES], dtype=np.int32)
    NON_EMPTY_FRAME_IDXS = np.full([N_SAMPLES, INPUT_SIZE], -1, dtype=np.float32)
    return X, y, NON_EMPTY_FRAME_IDXS

def save_split_data(X, y, NON_EMPTY_FRAME_IDXS, train_idxs, val_idxs):
    X_train, y_train, NON_EMPTY_FRAME_IDXS_TRAIN = X[train_idxs], y[train_idxs], NON_EMPTY_FRAME_IDXS[train_idxs]
    X_val, y_val, NON_EMPTY_FRAME_IDXS_VAL = X[val_idxs], y[val_idxs], NON_EMPTY_FRAME_IDXS[val_idxs]
    
    np.save('X_train.npy', X_train)
    np.save('y_train.npy', y_train)
    np.save('NON_EMPTY_FRAME_TRAIN.npy', NON_EMPTY_FRAME_IDXS_TRAIN)
    
    np.save('X_val.npy', X_val)
    np.save('y_val.npy', y_val)
    np.save('NON_EMPTY_FRAME_VAL.npy', NON_EMPTY_FRAME_IDXS_VAL)
    
    return X_train, y_train, X_val, y_val

def log_split_statistics(PARTICIPANT_IDS, train_idxs, val_idxs, X_train, y_train, X_val, y_val):
    print(f'Patient Train/Val: {set(PARTICIPANT_IDS[train_idxs]).intersection(PARTICIPANT_IDS[val_idxs])}')
    print(f'X_train shape: {X_train.shape}, X_val shape: {X_val.shape}')
    print(f'y_train shape: {y_train.shape}, y_val shape: {y_val.shape}')

def preprocess_data(N_SAMPLES, INPUT_SIZE, N_COLS, N_DIMS, train, SEED):
    X, y, NON_EMPTY_FRAME_IDXS = initialize_arrays(N_SAMPLES, INPUT_SIZE, N_COLS, N_DIMS)
    
    for row_idx, (file_path, sign_ord) in enumerate(tqdm(train[['file_path', 'ord_value']].values)):
        if row_idx % 5000 == 0:
            print(f'Generated {row_idx}/{N_SAMPLES}')
        
        data, non_empty_frame_idxs = get_data_preprocess(file_path)
        X[row_idx] = data
        y[row_idx] = sign_ord
        NON_EMPTY_FRAME_IDXS[row_idx] = non_empty_frame_idxs
        
        if np.isnan(data).sum() > 0:
            print(f'NaN values found at row {row_idx}')
            return data
    
    splitter = GroupShuffleSplit(test_size=0.20, n_splits=2, random_state=SEED)
    PARTICIPANT_IDS = train['participant_id'].values
    train_idxs, val_idxs = next(splitter.split(X, y, groups=PARTICIPANT_IDS))
    
    X_train, y_train, X_val, y_val = save_split_data(X, y, NON_EMPTY_FRAME_IDXS, train_idxs, val_idxs)
    
    log_split_statistics(PARTICIPANT_IDS, train_idxs, val_idxs, X_train, y_train, X_val, y_val)

preprocess_data(N_SAMPLES, INPUT_SIZE, N_COLS, N_DIMS, train, SEED)

In [ ]:
# import numpy as np
# from sklearn.model_selection import GroupShuffleSplit
# from tqdm import tqdm

# def initialize_arrays(N_SAMPLES, INPUT_SIZE, N_COLS, N_DIMS):
#     X = np.zeros([N_SAMPLES, INPUT_SIZE, N_COLS, N_DIMS], dtype=np.float32)
#     y = np.zeros([N_SAMPLES], dtype=np.int32)
#     NON_EMPTY_FRAME_IDXS = np.full([N_SAMPLES, INPUT_SIZE], -1, dtype=np.float32)
#     return X, y, NON_EMPTY_FRAME_IDXS

# def save_split_data(X, y, NON_EMPTY_FRAME_IDXS, train_idxs, val_idxs, test_idxs):
#     X_train, y_train, NON_EMPTY_FRAME_IDXS_TRAIN = X[train_idxs], y[train_idxs], NON_EMPTY_FRAME_IDXS[train_idxs]
#     X_val, y_val, NON_EMPTY_FRAME_IDXS_VAL = X[val_idxs], y[val_idxs], NON_EMPTY_FRAME_IDXS[val_idxs]
# #     X_test, y_test, NON_EMPTY_FRAME_IDXS_TEST = X[test_idxs], y[test_idxs], NON_EMPTY_FRAME_IDXS[test_idxs]
    
#     np.save('X_train.npy', X_train)
#     np.save('y_train.npy', y_train)
#     np.save('NON_EMPTY_FRAME_TRAIN.npy', NON_EMPTY_FRAME_IDXS_TRAIN)
    
#     np.save('X_val.npy', X_val)
#     np.save('y_val.npy', y_val)
#     np.save('NON_EMPTY_FRAME_VAL.npy', NON_EMPTY_FRAME_IDXS_VAL)
    
# #     np.save('X_test.npy', X_test)
# #     np.save('y_test.npy', y_test)
# #     np.save('NON_EMPTY_FRAME_TEST.npy', NON_EMPTY_FRAME_IDXS_TEST)
    
#     return X_train, y_train, X_val, y_val, X_test, y_test

# # def log_split_statistics(PARTICIPANT_IDS, train_idxs, val_idxs, test_idxs, X_train, y_train, X_val, y_val, X_test, y_test):
# #     print(f'Patient Train/Val/Test: {set(PARTICIPANT_IDS[train_idxs]).intersection(PARTICIPANT_IDS[val_idxs]).intersection(PARTICIPANT_IDS[test_idxs])}')
# #     print(f'X_train shape: {X_train.shape}, X_val shape: {X_val.shape}, X_test shape: {X_test.shape}')
# #     print(f'y_train shape: {y_train.shape}, y_val shape: {y_val.shape}, y_test shape: {y_test.shape}')

# def preprocess_data(N_SAMPLES, INPUT_SIZE, N_COLS, N_DIMS, train, SEED):
#     X, y, NON_EMPTY_FRAME_IDXS = initialize_arrays(N_SAMPLES, INPUT_SIZE, N_COLS, N_DIMS)
    
#     for row_idx, (file_path, sign_ord) in enumerate(tqdm(train[['file_path', 'ord_value']].values)):
#         if row_idx % 5000 == 0:
#             print(f'Generated {row_idx}/{N_SAMPLES}')
        
#         data, non_empty_frame_idxs = get_data_preprocess(file_path)
#         X[row_idx] = data
#         y[row_idx] = sign_ord
#         NON_EMPTY_FRAME_IDXS[row_idx] = non_empty_frame_idxs
        
#         if np.isnan(data).sum() > 0:
#             print(f'NaN values found at row {row_idx}')
#             return data
    
#     splitter = GroupShuffleSplit(test_size=0.20, n_splits=2, random_state=SEED)
#     PARTICIPANT_IDS = train['participant_id'].values
#     train_val_idxs, test_idxs = next(splitter.split(X, y, groups=PARTICIPANT_IDS))
    
#     splitter_val = GroupShuffleSplit(test_size=0.25, n_splits=2, random_state=SEED)
#     train_idxs, val_idxs = next(splitter_val.split(X[train_val_idxs], y[train_val_idxs], groups=PARTICIPANT_IDS[train_val_idxs]))
#     train_idxs = train_val_idxs[train_idxs]
#     val_idxs = train_val_idxs[val_idxs]
    
#     X_train, y_train, X_val, y_val, X_test, y_test = save_split_data(X, y, NON_EMPTY_FRAME_IDXS, train_idxs, val_idxs, test_idxs)
    
#     log_split_statistics(PARTICIPANT_IDS, train_idxs, val_idxs, test_idxs, X_train, y_train, X_val, y_val, X_test, y_test)

# preprocess_data(N_SAMPLES, INPUT_SIZE, N_COLS, N_DIMS, train, SEED)


# Load New Data from NPY file

In [23]:
def display_shapes_and_dtypes(data, names):
    for element, name in zip(data, names):
        print(f'{name} shape: {element.shape}, dtype: {element.dtype}')

In [ ]:
# # ROOT_DIR = '/kaggle/input/gisl-npy/gisl-preproc'
# ROOT_DIR = ''
# X_train = np.load(f'{ROOT_DIR}/X_train.npy')
# y_train = np.load(f'{ROOT_DIR}/y_train.npy')
# NON_EMPTY_FRAME_TRAIN = np.load(f'{ROOT_DIR}/NON_EMPTY_FRAME_TRAIN.npy')

# X_val = np.load(f'{ROOT_DIR}/X_val.npy')
# y_val = np.load(f'{ROOT_DIR}/y_val.npy')
# NON_EMPTY_FRAME_VAL = np.load(f'{ROOT_DIR}/NON_EMPTY_FRAME_VAL.npy')
# validation_data = ({ 'frames': X_val, 'non_empty_frame_idxs': NON_EMPTY_FRAME_VAL }, y_val)

# # X_test = np.load(f'{ROOT_DIR}/X_test.npy')
# # y_test = np.load(f'{ROOT_DIR}/y_test.npy')
# # NON_EMPTY_FRAME_TEST = np.load(f'{ROOT_DIR}/NON_EMPTY_FRAME_TEST.npy')
# # validation_data = ({ 'frames': X_test, 'non_empty_frame_idxs': NON_EMPTY_FRAME_TEST }, y_test)


# display_shapes_and_dtypes([X_train, y_train, NON_EMPTY_FRAME_TRAIN], ['X_train', 'y_train', 'NON_EMPTY_FRAME_TRAIN'])
# display_shapes_and_dtypes([X_val, y_val, NON_EMPTY_FRAME_VAL], ['X_val', 'y_val', 'NON_EMPTY_FRAME_VAL'])
# # display_shapes_and_dtypes([X_test, y_test, NON_EMPTY_FRAME_TEST], ['X_test', 'y_test', 'NON_EMPTY_FRAME_TEST'])

# print(f'# NaN Values X_train: {np.isnan(X_train).sum()}')
# print(f'# NaN Values X_val: {np.isnan(X_val).sum()}')
# # print(f'# NaN Values X_test: {np.isnan(X_test).sum()}')

In [24]:
ROOT_DIR = '/kaggle/input/gislr-dataset-public'
X_train = np.load(f'{ROOT_DIR}/X_train.npy')
X_train = X_train[:, :, :, :2]
y_train = np.load(f'{ROOT_DIR}/y_train.npy')
NON_EMPTY_FRAME_TRAIN = np.load(f'{ROOT_DIR}/NON_EMPTY_FRAME_IDXS_TRAIN.npy')
X_val = np.load(f'{ROOT_DIR}/X_val.npy')
X_val = X_val[:, :, :, :2]
y_val = np.load(f'{ROOT_DIR}/y_val.npy')
NON_EMPTY_FRAME_VAL = np.load(f'{ROOT_DIR}/NON_EMPTY_FRAME_IDXS_VAL.npy')
validation_data = ({ 'frames': X_val, 'non_empty_frame_idxs': NON_EMPTY_FRAME_VAL }, y_val)
display_shapes_and_dtypes([X_train, y_train, NON_EMPTY_FRAME_TRAIN], ['X_train', 'y_train', 'NON_EMPTY_FRAME_IDXS_TRAIN'])
display_shapes_and_dtypes([X_val, y_val, NON_EMPTY_FRAME_VAL], ['X_val', 'y_val', 'NON_EMPTY_FRAME_IDXS_VAL'])
print(f'# NaN Values X_train: {np.isnan(X_train).sum()}')

X_train shape: (80229, 64, 66, 2), dtype: float32
y_train shape: (80229,), dtype: int32
NON_EMPTY_FRAME_IDXS_TRAIN shape: (80229, 64), dtype: float32
X_val shape: (14248, 64, 66, 2), dtype: float32
y_val shape: (14248,), dtype: int32
NON_EMPTY_FRAME_IDXS_VAL shape: (14248, 64), dtype: float32
# NaN Values X_train: 0


# Normalisation

In [25]:
def get_lips_mean_std():
    # LIPS
    LIPS_MEAN_X = np.zeros([LIPS_IDXS.size], dtype=np.float32)
    LIPS_MEAN_Y = np.zeros([LIPS_IDXS.size], dtype=np.float32)
    LIPS_STD_X = np.zeros([LIPS_IDXS.size], dtype=np.float32)
    LIPS_STD_Y = np.zeros([LIPS_IDXS.size], dtype=np.float32)

    for col, ll in enumerate(tqdm( np.transpose(X_train[:,:,LIPS_IDXS], [2,3,0,1]).reshape([LIPS_IDXS.size, N_DIMS, -1]) )):
        for dim, l in enumerate(ll):
            v = l[np.nonzero(l)]
            if dim == 0: # X
                LIPS_MEAN_X[col] = v.mean()
                LIPS_STD_X[col] = v.std()
            if dim == 1: # Y
                LIPS_MEAN_Y[col] = v.mean()
                LIPS_STD_Y[col] = v.std()
    
    
    LIPS_MEAN = np.array([LIPS_MEAN_X, LIPS_MEAN_Y]).T
    LIPS_STD = np.array([LIPS_STD_X, LIPS_STD_Y]).T
    
    return LIPS_MEAN, LIPS_STD

LIPS_MEAN, LIPS_STD = get_lips_mean_std()

  0%|          | 0/40 [00:00<?, ?it/s]

In [26]:
def get_left_right_hand_mean_std():
    # LEFT HAND
    LEFT_HANDS_MEAN_X = np.zeros([LEFT_HAND_IDXS.size], dtype=np.float32)
    LEFT_HANDS_MEAN_Y = np.zeros([LEFT_HAND_IDXS.size], dtype=np.float32)
    LEFT_HANDS_STD_X = np.zeros([LEFT_HAND_IDXS.size], dtype=np.float32)
    LEFT_HANDS_STD_Y = np.zeros([LEFT_HAND_IDXS.size], dtype=np.float32)

    for col, ll in enumerate(tqdm( np.transpose(X_train[:,:,LEFT_HAND_IDXS], [2,3,0,1]).reshape([LEFT_HAND_IDXS.size, N_DIMS, -1]) )):
        for dim, l in enumerate(ll):
            v = l[np.nonzero(l)]
            if dim == 0: # X
                LEFT_HANDS_MEAN_X[col] = v.mean()
                LEFT_HANDS_STD_X[col] = v.std()
            if dim == 1: # Y
                LEFT_HANDS_MEAN_Y[col] = v.mean()
                LEFT_HANDS_STD_Y[col] = v.std()

    LEFT_HANDS_MEAN = np.array([LEFT_HANDS_MEAN_X, LEFT_HANDS_MEAN_Y]).T
    LEFT_HANDS_STD = np.array([LEFT_HANDS_STD_X, LEFT_HANDS_STD_Y]).T
    
    return LEFT_HANDS_MEAN, LEFT_HANDS_STD

LEFT_HANDS_MEAN, LEFT_HANDS_STD = get_left_right_hand_mean_std()

  0%|          | 0/21 [00:00<?, ?it/s]

In [27]:
def get_pose_mean_std():
    # POSE
    POSE_MEAN_X = np.zeros([POSE_IDXS.size], dtype=np.float32)
    POSE_MEAN_Y = np.zeros([POSE_IDXS.size], dtype=np.float32)
    POSE_STD_X = np.zeros([POSE_IDXS.size], dtype=np.float32)
    POSE_STD_Y = np.zeros([POSE_IDXS.size], dtype=np.float32)

    for col, ll in enumerate(tqdm( np.transpose(X_train[:,:,POSE_IDXS], [2,3,0,1]).reshape([POSE_IDXS.size, N_DIMS, -1]) )):
        for dim, l in enumerate(ll):
            v = l[np.nonzero(l)]
            if dim == 0: # X
                POSE_MEAN_X[col] = v.mean()
                POSE_STD_X[col] = v.std()
            if dim == 1: # Y
                POSE_MEAN_Y[col] = v.mean()
                POSE_STD_Y[col] = v.std()

    POSE_MEAN = np.array([POSE_MEAN_X, POSE_MEAN_Y]).T
    POSE_STD = np.array([POSE_STD_X, POSE_STD_Y]).T
    
    return POSE_MEAN, POSE_STD

POSE_MEAN, POSE_STD = get_pose_mean_std()

  0%|          | 0/5 [00:00<?, ?it/s]

# Samples

In [28]:
BATCH_ALL_SIGNS_N = 4

In [29]:
# Custom sampler to get a batch containing N times all signs
def get_train_batch_all_signs(X, y, NON_EMPTY_FRAME_IDXS, n=BATCH_ALL_SIGNS_N):
    # Arrays to store batch in
    X_batch = np.zeros([NUM_CLASS*n, INPUT_SIZE, N_COLS, N_DIMS], dtype=np.float32)
    y_batch = np.arange(0, NUM_CLASS, step=1/n, dtype=np.float32).astype(np.int64)
    non_empty_frame_idxs_batch = np.zeros([NUM_CLASS*n, INPUT_SIZE], dtype=np.float32)
    
    # Dictionary mapping ordinally encoded sign to corresponding sample indices
    CLASS2IDXS = {}
    for i in range(NUM_CLASS):
        CLASS2IDXS[i] = np.argwhere(y == i).squeeze().astype(np.int32)
            
    while True:
        # Fill batch arrays
        for i in range(NUM_CLASS):
            idxs = np.random.choice(CLASS2IDXS[i], n)
            X_batch[i*n:(i+1)*n] = X[idxs]
            non_empty_frame_idxs_batch[i*n:(i+1)*n] = NON_EMPTY_FRAME_IDXS[idxs]
        
        yield { 'frames': X_batch, 'non_empty_frame_idxs': non_empty_frame_idxs_batch }, y_batch

# Transformer Model

# Model

In [40]:
import tensorflow as tf
from tensorflow.keras.layers import *

In [41]:
gc.collect()

2638

In [42]:
do = 0.3

model = tf.keras.Sequential()
model.add(InputLayer(input_shape=(64, 66, 2)))
model.add(Reshape((64, 66*2)))

model.add(Conv1D(64, 1, strides=1, padding='valid', activation='relu'))
model.add(Conv1D(64, 1, strides=1, padding='valid', activation='relu'))
model.add(BatchNormalization())
model.add(DepthwiseConv1D(3, strides=1, padding='valid', depth_multiplier=1, activation='relu'))
model.add(BatchNormalization())

model.add(Conv1D(128, 1, strides=1, padding='valid', activation='relu'))
model.add(Conv1D(128, 1, strides=1, padding='valid', activation='relu'))
model.add(BatchNormalization())
model.add(DepthwiseConv1D(3, strides=1, padding='valid', depth_multiplier=1, activation='relu'))
model.add(BatchNormalization())

model.add(Conv1D(180, 1, strides=1, padding='valid', activation='relu'))
model.add(Conv1D(180, 1, strides=1, padding='valid', activation='relu'))
model.add(BatchNormalization())
model.add(DepthwiseConv1D(5, strides=2, padding='valid', depth_multiplier=4, activation='relu'))
model.add(BatchNormalization())

model.add(MaxPool1D(2, 2))
model.add(LSTM(64, return_sequences=True))

model.add(Conv1D(256, 1, strides=1, padding='valid', activation='relu'))
model.add(Conv1D(256, 1, strides=1, padding='valid', activation='relu'))
model.add(BatchNormalization())
model.add(DepthwiseConv1D(3, strides=1, padding='valid', depth_multiplier=1, activation='relu'))
model.add(BatchNormalization())

model.add(Conv1D(512, 1, strides=1, padding='valid', activation='relu'))
model.add(Conv1D(512, 1, strides=1, padding='valid', activation='relu'))
model.add(BatchNormalization())
model.add(DepthwiseConv1D(5, strides=2, padding='valid', depth_multiplier=4, activation='relu'))
model.add(BatchNormalization())

model.add(GlobalAvgPool1D())
model.add(Dropout(rate=do))

model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(rate=do))

model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(rate=do))

model.add(Dense(250, activation='softmax'))

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/input_layer.py:25: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [43]:
from tensorflow.keras.callbacks import LearningRateScheduler
import math

In [44]:
def step_decay(epoch):
    initial_lr = 0.001
    drop = 0.5
    epochs_drop = 10
    lr = initial_lr * math.pow(drop, math.floor((1 + epoch) / epochs_drop))
    return lr

# Create the LearningRateScheduler callback
lr_scheduler = LearningRateScheduler(step_decay)

In [45]:
model.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics = ['acc']
)

In [46]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    min_delta=0.001,
    restore_best_weights=True
)

In [47]:
tf.keras.backend.clear_session()

history = model.fit(
    X_train, y_train,
    epochs=N_EPOCHS,
    batch_size=64,
    validation_data=(X_val, y_val),
    verbose = 2,
    callbacks = [lr_scheduler, early_stopping]
)

Epoch 1/1000
1254/1254 - 51s - 40ms/step - acc: 0.0230 - loss: 5.1875 - val_acc: 0.0503 - val_loss: 4.4963 - learning_rate: 0.0010
Epoch 2/1000
1254/1254 - 33s - 26ms/step - acc: 0.0915 - loss: 4.2124 - val_acc: 0.0313 - val_loss: 5.4787 - learning_rate: 0.0010
Epoch 3/1000
1254/1254 - 33s - 26ms/step - acc: 0.1865 - loss: 3.5563 - val_acc: 0.0425 - val_loss: 5.5325 - learning_rate: 0.0010
Epoch 4/1000
1254/1254 - 33s - 26ms/step - acc: 0.2587 - loss: 3.1474 - val_acc: 0.2019 - val_loss: 3.4164 - learning_rate: 0.0010
Epoch 5/1000
1254/1254 - 33s - 26ms/step - acc: 0.3183 - loss: 2.8537 - val_acc: 0.0943 - val_loss: 5.4512 - learning_rate: 0.0010
Epoch 6/1000
1254/1254 - 33s - 26ms/step - acc: 0.3642 - loss: 2.6393 - val_acc: 0.1289 - val_loss: 4.2982 - learning_rate: 0.0010
Epoch 7/1000
1254/1254 - 33s - 26ms/step - acc: 0.3966 - loss: 2.4831 - val_acc: 0.1347 - val_loss: 4.3763 - learning_rate: 0.0010
Epoch 8/1000
1254/1254 - 33s - 26ms/step - acc: 0.4239 - loss: 2.3591 - val_acc: 0.

KeyboardInterrupt: 